<a href="https://colab.research.google.com/github/dardeshna/frc-stuffs/blob/master/linalg/opr_2022cabl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Offensive Power Rating (OPR)

Offensive Power Rating (OPR) is a metric to estimate a team's average point contribution to their alliance. We can calculate OPR using the match scores from an event.

In [ ]:
# setup
import numpy as np
import pandas as pd
import requests
import json

## Pulling data from TBA

The first step is to pull event data from [The Blue Alliance](https://www.thebluealliance.com/). We can use their Read API which is documented here: https://www.thebluealliance.com/apidocs/v3

To calculate OPR we will need a list of teams and scores for each match. We can also pull OPR from TBA for comparison.

In [ ]:
# generate a TBA auth key at https://www.thebluealliance.com/account
auth_key = ''

event = '2022cabl' # beach blitz 2022

# get a list of teams
teams_response = requests.get(f'https://www.thebluealliance.com/api/v3/event/{event}/teams/keys', headers={
    'X-TBA-Auth-Key': auth_key
})
# get match data
match_response = requests.get(f'https://www.thebluealliance.com/api/v3/event/{event}/matches/simple', headers={
    'X-TBA-Auth-Key': auth_key
})
# get opr calculated by TBA
oprs_response = requests.get(f'https://www.thebluealliance.com/api/v3/event/{event}/oprs', headers={
    'X-TBA-Auth-Key': auth_key
})

## Processing the data

Next we need to process the responses obtained by the TBA API. We can convert them to nested python lists and dictionaries using the `.json()` function. OPR is traditionally calculated using only quals matches, so we also want to filter by match type.

In [ ]:
# get a list of team names
teams = teams_response.json()

# get a list of dictionaries with data for each match
matches = match_response.json()
qual_matches = [m for m in matches if m['comp_level']=='qm'] # select only quals matches

# print the first 10 teams
print("teams:", teams[:10])

# print qm1 data
print("qm1:", json.dumps(qual_matches[0], indent=4))

teams: ['frc1138', 'frc1160', 'frc1197', 'frc1836', 'frc2122', 'frc2637', 'frc2658', 'frc2659', 'frc294', 'frc3128']
qm1: {
    "actual_time": null,
    "alliances": {
        "blue": {
            "dq_team_keys": [],
            "score": 61,
            "surrogate_team_keys": [],
            "team_keys": [
                "frc7230",
                "frc294",
                "frc1138"
            ]
        },
        "red": {
            "dq_team_keys": [],
            "score": 97,
            "surrogate_team_keys": [],
            "team_keys": [
                "frc3476",
                "frc6560",
                "frc6220"
            ]
        }
    },
    "comp_level": "qm",
    "event_key": "2022cabl",
    "key": "2022cabl_qm1",
    "match_number": 1,
    "predicted_time": 1667666700,
    "set_number": 1,
    "time": 1667666700,
    "winning_alliance": "red"
}


## Calculating OPR
OPR just solves a system of equations to estimate each team's average point contribution. Specifically, for match $i$ we have the following equations:

\begin{gather*}
x_{{r_i}_a}+x_{{r_i}_b}+x_{{r_i}_c}=s_{r_i} \\
x_{{b_i}_a}+x_{{b_i}_b}+x_{{b_i}_c}=s_{b_i} \\
\end{gather*}

Here, $x_{{r_i}_a}$ represents the OPR of the first team on the red alliance in match $i$. Similarly, $s_{r_i}$ is the red alliance's score in match $i$.

To be clear, these symbols are just placeholders. If teams 8, 254, and 1678 scored 133 points for the red alliance while 971, 973, and 1323 scored 128 points for blue, we would have:

\begin{gather*}
x_{8}+x_{254}+x_{1678}=133 \\
x_{971}+x_{973}+x_{1323}=128
\end{gather*}

For $m$ matches, we can stack the two equations from each match to get a total of $2m$ equations:

\begin{gather*}
x_{{r_1}_a}+x_{{r_1}_b}+x_{{r_1}_c}=s_{r_1} \\
x_{{b_1}_a}+x_{{b_1}_b}+x_{{b_1}_c}=s_{b_1} \\
x_{{r_2}_a}+x_{{r_2}_b}+x_{{r_2}_c}=s_{r_2} \\
x_{{b_2}_a}+x_{{b_2}_b}+x_{{b_2}_c}=s_{b_2} \\
\vdots \\
x_{{r_m}_a}+x_{{r_m}_b}+x_{{r_m}_c}=s_{r_m} \\
x_{{b_m}_a}+x_{{b_m}_b}+x_{{b_m}_c}=s_{b_m} \\
\end{gather*}

These equations can be written in matrix form:

\begin{gather*}
\begin{bmatrix}
x_{{r_1}_a}+x_{{r_1}_b}+x_{{r_1}_c} \\
x_{{b_1}_a}+x_{{b_1}_b}+x_{{b_1}_c} \\
x_{{r_2}_a}+x_{{r_2}_b}+x_{{r_2}_c} \\
x_{{b_2}_a}+x_{{b_2}_b}+x_{{b_2}_c} \\
\vdots \\
x_{{r_m}_a}+x_{{r_m}_b}+x_{{r_m}_c} \\
x_{{b_m}_a}+x_{{b_m}_b}+x_{{b_m}_c}
\end{bmatrix}
= \begin{bmatrix} s_{r_1} \\ s_{b_1} \\ s_{r_2} \\ s_{b_2} \\ \vdots \\ s_{r_m} \\ s_{b_m} \end{bmatrix}
\end{gather*}

If we have $n$ teams at the event, we can break up the left-hand side into the product of $\mathbf{A} \in \mathbb{R}^{2m \times n}$ and $\mathbf{x} \in \mathbb{R}^{n}$. The entries of $\mathbf{x}$ contains the OPRs for each team, while each row of $\mathbf{A}$ is zero except for the columns corresponding to the teams on that alliance. After doing so, we end up with something like this:

\begin{gather*}
\mathbf{Ax} =
\begin{bmatrix}
1 & 0 & 0 & 1 & 0 & 1 & \dots & 0 \\
0 & 1 & 1 & 0 & 1 & 1 & \dots & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & \dots & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & \dots & 0 \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & 0 & 0 & 0 & \dots & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & \dots & 0
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6 \\ \vdots \\x_n \\
\end{bmatrix}
= \begin{bmatrix} s_{r_1} \\ s_{b_1} \\ s_{r_2} \\ s_{b_2} \\ \vdots \\ s_{r_m} \\ s_{b_m} \end{bmatrix}
= \mathbf{b}
\end{gather*}

At a typical FRC event, $2m > n$ so we have more equations than unknowns and $\mathbf{A}$ is a tall matrix. This means that while there is no exact solution for $\mathbf{x}$, we can use `np.linalg.lstsq(A, b)` to solve for an $\mathbf{x}$ that minimizes the error $\| \mathbf{Ax}-\mathbf{b} \|$.


In [ ]:
# initialize matrices
A = np.zeros((2*len(qual_matches), len(teams)))
b = np.zeros((2*len(qual_matches),))

# iterate over matches
for i, qm in enumerate(qual_matches):

    # fill in equation for red alliance (row 2*i)
    for team in qm['alliances']['red']['team_keys']:
        A[2*i, teams.index(team)] = 1
    b[2*i] = qm['alliances']['red']['score']
    
    # fill in equation for blue alliance (row 2*i+1)
    for team in qm['alliances']['blue']['team_keys']:
        A[2*i+1, teams.index(team)] = 1
    b[2*i+1] = qm['alliances']['blue']['score']

# solve for OPRs
x = np.linalg.lstsq(A, b, rcond=None)[0]

## Comparison to TBA OPR

Finally, we can sort teams by OPR and compare our values to TBA's values!

In [ ]:
# get dictionary of OPRs from TBA
tba_opr = oprs_response.json()['oprs']

# build a new dictionary with both our OPR and TBA's OPR
data = {}
for team, opr in zip(teams, x):
    data[team] = {'calculated_opr': opr, 'tba_opr': tba_opr[team]}

# convert dict to a pandas dataframe and sort by our OPR
pd.DataFrame.from_dict(data, orient='index').sort_values('calculated_opr', ascending=False)

,calculated_opr,tba_opr
frc7157,54.726026,54.726026
frc5199,52.236961,52.236961
frc696,48.965110,48.965110
frc8,39.492574,39.492574
frc3255,39.368547,39.368547
frc6995,38.331170,38.331170
frc3476,37.868540,37.868540
frc3128,36.634645,36.634645
frc2637,33.493575,33.493575
frc6560,33.211455,33.211455
